<table>
 <tr align=left><td><img align=left src="./images/CC-BY.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli</td>
</table>

In [ ]:
from __future__ import print_function

%matplotlib inline
import numpy
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML

import os
import pyclaw
path = os.path.join(os.environ.get("CLAW", os.getcwd()), "pyclaw", "fvmbook", "chap6")
os.chdir(path)
import advection

# Boundary Conditions

Up until now we have yet to address a way to enfore boundary conditions on our methods.  In this lecture we will address this and study how we might handle both simple and complex boundary conditions.

The primary way we will implement boundary conditions is by adding **ghost cells** to the domain.  These cells are outside of the domain and can be set arbitrarily to enforce many types of boundary conditions depending on what is desired.  Keep in mind a couple of things though:

1. For many hyperbolic PDEs we only need BCs on one end of the domain.
1. Enforcing non-reflecting BCs is important for many applications is highly non-trivial.
1. Setting BCs may require the solution of more complex equations so that an incoming boundary condition is correctly set.

The rest of this lecture will use examples to illustrate different BCs.

## Example:  Advection

Consider the scalar advection equation $q_t + u q_x$ on $x\in[a,b]$ with BC
$$
    q(a, t) = g_a(t)
$$
that can easily be generalized to also be at $x=b$.  Note also that we only have one BC needed.

### Outflow Boundary Conditions

Analytically we have no BC at $x = b$ and if we simply use an upwind method we do not need one numerically.  However, if we are using a Lax-Wendroff like method we still require a value at $x=b$ to be specified.  One of the easiest ways to specify this is to use **zero-order extrapolation** boundary conditions defined by
$$
    Q^n_{N+1} = Q^n_N \quad Q^n_{N+2} = Q^n_N.
$$
This formulation then implies that the gradient outside of the domain is zero and that in most cases any wave approaching the boundary will simply propagate out of the domain.

What about the case where we may want to use a first-order extrapolation given by
$$
    Q^n_{N+1} = Q^n_N + (Q^n_N - Q^n_{N-1} ) = 2 Q^n_{N} - Q^n_{N-1}?
$$
This leads to $\Delta Q^n_{N+1/2} = \Delta Q^n_{N-1/2}$ and the correction terms cancel leading to an update that again reduces to first-order upwind.

This extrapolation seems simplistic but turns out to be extremely powerful.  This extrapolation, in conjunction with our finite volume methods and characteristic decomposition, tend to be very robust when considering non-reflecting boundary conditions.  In fact first-order extrapolation can even lead to issues not found in the zero-order extrapolation and is generally not used.

### Inflow Boundary Conditions

At the inflow boundary it is a bit more clear what to do.  Depending on how the BC is specified there are multiple ways to numerically represent the BC.  The most straight forward perhaps is the case when a direct flux is specified that would easily integrate into the methods discussed already.

Based off of this we could want to integrate along the boundary such that
$$\begin{aligned}
    F^n_{1/2} &= \frac{1}{\Delta t} \int^{t_{n+1}}_{t_n} u q(a, t) dt \\
    &=\frac{u}{\Delta t} \int^{t_{n+1}}_{t_n} g_a(t) dt
\end{aligned}$$
or
$$
    F^n_{1/2} = u g_a(t_n + \Delta t / 2)
$$
as an approximation.

One draw back to the flux integration approach is that we may need to have fluxes also specified further from the boundary for larger stencil methods.

What if instead we tried to find a way to directly specify the values for $Q$ in the ghost cells?  This allows the method that we are using interior to the domain to be applied everywhere where we have two cells sharing a boundary.  For instance
$$
    Q^n_0 = \frac{1}{\Delta x} \int^a_{a - \Delta x} q(x, t_n) dx.
$$
The drawback here is that we do not know in general $q(x, t_n)$.

Instead we can use the fact we know the characteristics as they head into the domain.  We could for instance use
$$\begin{aligned}
    q(x, t_n) &= q(a, t_n + (a -x) / u)\\
    &= g_a(t_n + (a - x) / u)
\end{aligned}$$
leading to
$$\begin{aligned}
    Q^n_0 &= \frac{1}{\Delta x} \int^a_{a - \Delta x} g_a\left( t_n + \frac{a - x}{u} \right ) dx \\
    &= \frac{u}{\Delta x} \int^{t_n + \Delta x / u}_{t_n} g_a(\tau) d\tau
\end{aligned}$$
We can again approximate these integrals with higher-order approximations.

## Example:  Acoustics

Systems of equations are slightly more complex to specify BCs for but follow a similar pattern.  There are also special types of BCs that can arise for systems for which we will discuss.  We will use the linear acoustics equations for this discussion.

Recall that the acoustics equations linearized around a state where $u=0$ takes the form
$$\begin{aligned}
    &p_t + K_0 u_x = 0 \\
    \rho_0 &u_t + p_x = 0
\end{aligned}$$
and has characteristic variables
$$\begin{aligned}
    w^1(x,t) &= \frac{1}{2 Z_0} (-p + Z_0 u) \\
    w^2(x,t) &= \frac{1}{2 Z_0} (p + Z_0 u)
\end{aligned}$$

### Non-Reflecting Boundary Conditions

**Non-reflecting boundary conditions**, also called **absorbing boundary conditions**, are useful when you would like to have waves that leave a finite domain to not impact what is going on interior to the domain.  One good example of this is when you want an infinite domain, such as the case with a Cauchy problem, but of course cannot represent this in a numerical example.  The non-reflecting boundary conditions will preserve the steady state solution that would evolve between say waves moving in opposite directions.  Let us study this in the context of the acoustics equations.

With Godunov-type methods that solve a Riemann problem it turns out that zero-order extrapolation works well as non-reflecting boundary conditions.  One way to see that this is indeed a good idea is to consider the update formulas in terms of the characteristic variables:
$$
    W^1 = \frac{-Q^1 + Z_0 Q^2}{2 Z_0} \\
    W^2 = \frac{Q^1 + Z_0 Q^2}{2 Z_0}.
$$
This separation into characteristic fields then has the same properties as the scalar advection case.  This also leads to the conclusion that
$$
    Q^n_0 = Q^n_1 \quad \quad Q^n_{-1} = Q^n_1.
$$
Note that this causes the gradient to be zero and therefore the flux.

### Incoming Waves

Say we want to impose the boundary condition
$$
    w^2(a, t) = \sin(\omega t)
$$
without any reflection of waves hitting this boundary.  Decomposing $Q_1$ into the characteristic variables such that
$$
    Q_1 = W^1_1 r^1 + W^2_1 r^2
$$
then the ghost cell should be set to
$$
    Q_0 = W^1_1 r^1 + \sin(\omega(t_n + \Delta x / 2 c_0)) r^2.
$$

If we want to generalize this to a system of equations with the boundary condition of $g_a(t)$ then we can also write
$$
    Q_0 = Q_1 + \left [g_a(t_n + \Delta x / 2 \lambda^j) - W^j_1\right ] r^j.
$$

### Solid Walls

A particular type of boundary condition comes up often in fluid dynamics, that of a solid wall.  If we consider a tube of gas that has a wall at one end we would expect a sound wave to reflect off of the end of the tube with the wall and reverse its direction.  The effective solid wall boundary condition should be then
$$
    u(a, t) = 0.
$$

The key to enforcing a wall boundary condition is to observe that if we set the ghost cell value to an equivalent value for the pressure but an opposite $u$ that this will satisfy the above condition.  In other words
$$
    p^0(a - \xi) = p^0(a + \xi) \\
    u^0(a - \xi) = -u^0(a + \xi)
$$

This then suggests that we set the ghost cells to
$$
    Q_0 = \begin{bmatrix} p_0 \\ u_0 \end{bmatrix} = \begin{bmatrix} p_1 \\ -u_1 \end{bmatrix} \\
    Q_{-1} = \begin{bmatrix} p_{-1} \\ u_{-1} \end{bmatrix} = \begin{bmatrix} p_2 \\ -u_2 \end{bmatrix}
$$

### Oscillating Walls

Instead of a solid wall we may want to specify a wall that is oscillating.  This could represent a source of acoustic waves, such as a speaker.  Here we will only consider small amplitude motions.

Consider again the fixed domain $x \in [a, b]$ with the boundary condition
$$
    u(a, t) = U(t).
$$
Note that this will degenerate into our solid wall case if $U(t) = 0$.

Consider a single frequency oscillation prescribed by
$$
    U(t) = \epsilon \sin(\omega t) \quad \epsilon \ll 1.
$$
Using what we had from before to prescribe the boundary condition we could then specify
$$
    Q_0 = \begin{bmatrix} p_0 \\ u_0 \end{bmatrix} = \begin{bmatrix} p_1 \\ 2 U(t_n) - u_1 \end{bmatrix} \\
    Q_{-1} = \begin{bmatrix} p_{-1} \\ u_{-1} \end{bmatrix} = \begin{bmatrix} p_2 \\ 2 U(t_n)-u_2 \end{bmatrix}
$$

## Periodic Boundary Conditions

Last but not least we also have periodic BCs to cover.  This is perhaps the easiest of the BCs to specify where with our previous definitions set
$$
    Q^n_{-1} = Q^n_{N-1} \quad Q^n_0 = Q^n_N \quad Q^n_{N+1} = Q^n_1 \quad Q^n_{N+2} Q^n_2.
$$